In [ ]:
features_selected = ['department', 'region', 'awards_won?', 
                     'previous_year_rating', 'KPIs_met >80%',
                      'avg_training_score', 'total_score', 'pca_combined']
                      

In [ ]:
features_selected_categorical = ['department', 'region', 'awards_won?','KPIs_met >80%']
features_selected_ordinal = ['previous_year_rating']
features_selected_numerical = ['avg_training_score', 'total_score', 'pca_combined']